## 📊 How to Monitor Training Progress

Detectron2 provides multiple ways to track training:

### **Method 1: TensorBoard (Real-time Visualization)**
Open a terminal and run:
```bash
tensorboard --logdir=./outputs
```
Then open http://localhost:6006 in your browser to see live loss curves, learning rate, etc.

### **Method 2: Jupyter Output (Console Logs)**
Detectron2 automatically prints training status. You'll see messages like:
```
[01/13 10:23:45] d2.engine.train_loop WARNING: ...
[01/13 10:23:50] d2.engine.train_loop INFO: [Iteration 50/1000] ...
````

### **Method 3: Check Training Files**
Training checkpoints are saved to `./outputs/` automatically. Check file modification times:
```bash
ls -lh ./outputs/model_*.pth  # See if new checkpoints appear
```

### **If Training Seems "Hung":**
1. Check if the GPU is being used: `nvidia-smi` (in terminal)
2. Look for the log file: `./outputs/log.txt`
3. Check if a checkpoint file is being created (means training is working!)
4. Wait at least 1-2 minutes before concluding it's hung

In [ ]:
# Quick Reference: What You'll See During Training

print("""
╔════════════════════════════════════════════════════════════════════════════╗
║                   TRAINING PROGRESS INDICATORS                             ║
╚════════════════════════════════════════════════════════════════════════════╝

When you run trainer.train(), Detectron2 will output messages like:

📋 LOG EXAMPLE:
───────────────
[01/13 10:23:45] d2.engine.train_loop WARNING: Using 0 GPUs
[01/13 10:24:15] d2.engine.train_loop INFO: [Iteration 50/1000] total_loss: 2.345
[01/13 10:24:45] d2.engine.train_loop INFO: [Iteration 100/1000] total_loss: 1.987
[01/13 10:25:15] d2.engine.train_loop INFO: [Iteration 150/1000] total_loss: 1.654
...

🔍 WHAT EACH PART MEANS:
────────────────────────
• [01/13 10:24:15]     = Date/time stamp
• d2.engine.train_loop = Detectron2 module name
• Iteration 50/1000    = Current iteration out of max
• total_loss: 2.345    = Current training loss (should decrease over time!)

📊 LOSS TRENDS:
───────────────
✅ GOOD:   2.5 → 2.0 → 1.5 → 1.0  (loss decreasing = training working!)
❌ BAD:    2.5 → 2.4 → 2.5 → 2.6   (loss not changing = check config)
❌ BAD:    NaN or Inf              (data/model issue = check inputs)

⏱️ TIMING:
──────────
• First few iterations are slower (GPU warmup, data load)
• After warmup, speed stabilizes to ~1-10 iterations/sec (depending on GPU)
• For 1000 iterations at 2 iter/sec = ~8 minutes

🎯 QUICK CHECKS TO CONFIRM TRAINING IS NOT HUNG:
─────────────────────────────────────────────────

1️⃣ CHECK GPU USAGE:
   Terminal: nvidia-smi
   Look for: python process using memory

2️⃣ CHECK FILES CREATED:
   Terminal: ls -lh ./outputs/
   Look for: model_*.pth files with recent timestamps

3️⃣ CHECK LOG FILE:
   Terminal: tail -f ./outputs/log.txt
   Look for: Recent timestamps and loss values

4️⃣ LOOK AT TERMINAL OUTPUT:
   The notebook should print "Iteration X/Y" messages regularly


🛑 COMMON ISSUES:
─────────────────

Issue: No output for 5+ minutes
Solution:
  • First minibatch might load slowly (~1-2 min on first run)
  • Check GPU usage with nvidia-smi
  • Check if process is running (ps aux | grep python)

Issue: "CUDA out of memory"
Solution:
  • Reduce cfg.SOLVER.IMS_PER_BATCH (currently = 4)
  • Reduce image size (currently = 512x512)

Issue: Loss is NaN
Solution:
  • Check data pipeline (run test cells 25-26 first!)
  • Reduce learning rate
  • Check for corrupted images

Issue: Loss not decreasing
Solution:
  • Training is still warming up (first 50-100 iterations)
  • Learning rate might be too high/low
  • Dataset might be too small

═════════════════════════════════════════════════════════════════════════════

Next: Run the training cell above! You should see output like the LOG EXAMPLE.
""")

print("\n✅ Press the cells above to start training.")
print("   You'll see real-time progress in the output!")

### 🔍 Test Wrapper: Understanding `build_train_loader` Step-by-Step

This cell allows you to **inspect the data transformation** without running full training.

**What it does:**
- Takes a small batch from your dataset
- Shows the transformation at each step
- Prints shapes, types, and values
- Visualizes the final Detectron2 format

**Run this cell independently** to understand the data pipeline!

In [ ]:
"""
🧪 TEST WRAPPER: Step-by-Step Data Transformation Inspector
============================================================

This test wrapper helps you understand what happens inside build_train_loader
without running full training. Run this cell to see the data flow!
"""

from detectron2.structures import Instances, Boxes, BitMasks
import matplotlib.pyplot as plt

def test_collate_function():
    """
    Test wrapper to inspect the collate_fn_batches transformation
    """
    print("="*80)
    print("🔍 DATA TRANSFORMATION TEST - STEP BY STEP")
    print("="*80)

    # =====================================================
    # STEP 1: Get a small batch from train_dataset
    # =====================================================
    print("\n📦 STEP 1: Loading raw batch from train_dataset")
    print("-" * 80)

    batch_size = 2  # Small batch for inspection
    batch = [train_dataset[i] for i in range(batch_size)]

    print(f"✓ Loaded {len(batch)} samples")
    print(f"  Each sample is a tuple: (image_tensor, mask_tensor)")
    print(f"\n  Sample 0 structure:")
    print(f"    - image type: {type(batch[0][0])}")
    print(f"    - image shape: {batch[0][0].shape if hasattr(batch[0][0], 'shape') else 'N/A'}")
    print(f"    - mask type: {type(batch[0][1])}")
    print(f"    - mask shape: {batch[0][1].shape if hasattr(batch[0][1], 'shape') else 'N/A'}")

    # =====================================================
    # STEP 2: Convert tensors to numpy
    # =====================================================
    print("\n\n🔄 STEP 2: Converting tensors to numpy arrays")
    print("-" * 80)

    image, mask = batch[0]  # Take first sample

    print(f"Before conversion:")
    print(f"  image: {type(image)}, shape={image.shape if hasattr(image, 'shape') else 'N/A'}")
    print(f"  mask: {type(mask)}, shape={mask.shape if hasattr(mask, 'shape') else 'N/A'}")

    if isinstance(image, torch.Tensor):
        image = image.cpu().numpy()
    if isinstance(mask, torch.Tensor):
        mask = mask.cpu().numpy()

    print(f"\nAfter conversion:")
    print(f"  image: {type(image)}, shape={image.shape}, dtype={image.dtype}")
    print(f"  mask: {type(mask)}, shape={mask.shape}, dtype={mask.dtype}")

    # =====================================================
    # STEP 3: Transpose image [C, H, W] → [H, W, C]
    # =====================================================
    print("\n\n📐 STEP 3: Transposing image for Detectron2 format")
    print("-" * 80)

    print(f"Before transpose: {image.shape} (channels first)")

    if image.ndim == 3 and image.shape[0] in [1, 3]:
        image = np.transpose(image, (1, 2, 0))

    print(f"After transpose: {image.shape} (channels last)")
    print(f"  This is the format Detectron2 expects: [Height, Width, Channels]")

    # =====================================================
    # STEP 4: Convert to uint8 (0-255 range)
    # =====================================================
    print("\n\n🎨 STEP 4: Converting to uint8 format (0-255)")
    print("-" * 80)

    print(f"Before conversion:")
    print(f"  dtype: {image.dtype}, min={image.min():.4f}, max={image.max():.4f}")

    if image.max() <= 1.0:
        image = (image * 255).astype(np.uint8)
        print(f"\nAfter conversion (was normalized 0-1):")
    else:
        image = image.astype(np.uint8)
        print(f"\nAfter conversion (was already 0-255):")

    print(f"  dtype: {image.dtype}, min={image.min()}, max={image.max()}")

    # =====================================================
    # STEP 5: Convert semantic mask to instance format
    # =====================================================
    print("\n\n🎭 STEP 5: Converting semantic segmentation to instance format")
    print("-" * 80)

    h, w = mask.shape
    print(f"Mask shape: {mask.shape} (H={h}, W={w})")
    print(f"Unique classes in mask: {np.unique(mask)}")

    instances = Instances((h, w))
    print(f"✓ Created empty Instances object with image_size=({h}, {w})")

    # Find unique classes (excluding background)
    sem_seg = torch.from_numpy(mask.astype(np.int32))
    unique_classes = torch.unique(sem_seg)
    unique_classes = unique_classes[unique_classes > 0]  # Skip background (class 0)

    print(f"\nClasses to process (excluding background): {unique_classes.tolist()}")

    # =====================================================
    # STEP 6: Extract masks and bounding boxes per class
    # =====================================================
    print("\n\n📦 STEP 6: Extracting per-class masks and bounding boxes")
    print("-" * 80)

    class_masks = []
    class_labels = []
    boxes_list = []

    for idx, class_id in enumerate(unique_classes):
        print(f"\nProcessing class {int(class_id)}:")

        # Create binary mask for this class
        class_mask = (sem_seg == class_id).numpy().astype(np.uint8)
        print(f"  Binary mask: {class_mask.shape}, pixels={class_mask.sum()}")

        # Find bounding box
        rows = np.any(class_mask, axis=1)
        cols = np.any(class_mask, axis=0)

        if rows.any() and cols.any():
            y_min, y_max = np.where(rows)[0][[0, -1]]
            x_min, x_max = np.where(cols)[0][[0, -1]]

            if x_max > x_min and y_max > y_min:
                box = [float(x_min), float(y_min), float(x_max + 1), float(y_max + 1)]
                print(f"  Bounding box: x=[{x_min}, {x_max}], y=[{y_min}, {y_max}]")
                print(f"  Box area: {(x_max - x_min) * (y_max - y_min)} pixels")

                class_masks.append(torch.from_numpy(class_mask.astype(np.float32)))
                class_labels.append(int(class_id))
                boxes_list.append(box)

    print(f"\n✓ Extracted {len(class_masks)} valid instances")

    # =====================================================
    # STEP 7: Wrapping in Detectron2 structures
    # =====================================================
    print("\n\n🎁 STEP 7: Wrapping in Detectron2 structures")
    print("-" * 80)

    if class_masks:
        print(f"Creating structures for {len(class_masks)} instances:")

        # Stack masks and wrap in BitMasks
        masks_tensor = torch.stack(class_masks)
        print(f"  1. Masks tensor shape: {masks_tensor.shape} (N, H, W)")

        instances.gt_masks = BitMasks(masks_tensor)
        print(f"  2. ✓ Wrapped in BitMasks: {type(instances.gt_masks)}")
        print(f"     - Has crop_and_resize: {hasattr(instances.gt_masks, 'crop_and_resize')}")

        instances.gt_classes = torch.tensor(class_labels, dtype=torch.int64)
        print(f"  3. ✓ Classes: {instances.gt_classes.tolist()}")

        instances.gt_boxes = Boxes(torch.tensor(boxes_list, dtype=torch.float32))
        print(f"  4. ✓ Boxes shape: {instances.gt_boxes.tensor.shape}")
    else:
        print("No instances found - creating empty structures")
        instances.gt_masks = BitMasks(torch.zeros((0, h, w), dtype=torch.float32))
        instances.gt_classes = torch.zeros(0, dtype=torch.int64)
        instances.gt_boxes = Boxes(torch.zeros((0, 4), dtype=torch.float32))

    # =====================================================
    # STEP 8: Final Detectron2 format
    # =====================================================
    print("\n\n✅ STEP 8: Final Detectron2 format")
    print("-" * 80)

    # Prepare image tensor
    image_tensor = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0

    sample_dict = {
        "image": image_tensor,
        "instances": instances
    }

    print(f"Final dictionary structure:")
    print(f"  'image':")
    print(f"    - type: {type(sample_dict['image'])}")
    print(f"    - shape: {sample_dict['image'].shape}")
    print(f"    - dtype: {sample_dict['image'].dtype}")
    print(f"    - range: [{sample_dict['image'].min():.4f}, {sample_dict['image'].max():.4f}]")

    print(f"\n  'instances':")
    print(f"    - type: {type(sample_dict['instances'])}")
    print(f"    - image_size: {sample_dict['instances'].image_size}")
    print(f"    - has gt_masks: {sample_dict['instances'].has('gt_masks')}")
    print(f"    - has gt_classes: {sample_dict['instances'].has('gt_classes')}")
    print(f"    - has gt_boxes: {sample_dict['instances'].has('gt_boxes')}")

    if sample_dict['instances'].has('gt_masks'):
        print(f"    - num_instances: {len(sample_dict['instances'])}")
        print(f"    - gt_masks type: {type(sample_dict['instances'].gt_masks)}")
        print(f"    - gt_classes: {sample_dict['instances'].gt_classes.tolist()}")

    # =====================================================
    # STEP 9: Visualization
    # =====================================================
    print("\n\n🎨 STEP 9: Visualization")
    print("-" * 80)

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Original image
    axes[0].imshow(image)
    axes[0].set_title("Original Image\n(uint8, [H, W, C])")
    axes[0].axis('off')

    # Semantic mask
    axes[1].imshow(mask, cmap='tab20', vmin=0, vmax=12)
    axes[1].set_title(f"Semantic Mask\n{len(unique_classes)} classes")
    axes[1].axis('off')

    # Bounding boxes overlay
    axes[2].imshow(image)
    if instances.has('gt_boxes'):
        for box, label in zip(instances.gt_boxes.tensor, instances.gt_classes):
            x1, y1, x2, y2 = box.tolist()
            rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                fill=False, edgecolor='red', linewidth=2)
            axes[2].add_patch(rect)
            axes[2].text(x1, y1-5, f'Class {int(label)}',
                        color='red', fontsize=10, weight='bold',
                        bbox=dict(facecolor='white', alpha=0.7))
    axes[2].set_title(f"Instance Boxes\n{len(instances)} instances")
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()

    print("\n" + "="*80)
    print("✅ TEST COMPLETE!")
    print("="*80)
    print("\nKey Takeaways:")
    print("  1. Raw data: PyTorch tensors [C, H, W], normalized 0-1")
    print("  2. Detectron2 needs: numpy [H, W, C], uint8 0-255")
    print("  3. Semantic masks → Instance format (per-class masks + boxes)")
    print("  4. BitMasks wrapper enables Detectron2's internal operations")
    print("  5. Final format: dict with 'image' tensor and 'instances' object")

    return sample_dict

# Run the test
print("🚀 Running data transformation test...\n")
test_sample = test_collate_function()

In [ ]:
"""
🧪 BONUS: Test the actual collate_fn_batches function
======================================================

This tests the EXACT function used in build_train_loader with a batch.
"""

def test_actual_collate_function():
    """
    Test the actual collate function that will be used during training
    """
    from detectron2.structures import Instances, Boxes, BitMasks

    print("="*80)
    print("🔬 TESTING ACTUAL COLLATE FUNCTION (with batch)")
    print("="*80)

    # This is the EXACT collate function from build_train_loader
    def collate_fn_batches(batch):
        """The actual collate function used in DataLoader"""
        batch_data = []
        for image, mask in batch:
            # Convert tensor to numpy if needed
            if isinstance(image, torch.Tensor):
                image = image.cpu().numpy()
            if isinstance(mask, torch.Tensor):
                mask = mask.cpu().numpy()

            # Convert [C, H, W] image to [H, W, C] for Detectron2
            if image.ndim == 3 and image.shape[0] in [1, 3]:
                image = np.transpose(image, (1, 2, 0))

            # Ensure uint8 format (0-255)
            if image.max() <= 1.0:
                image = (image * 255).astype(np.uint8)
            else:
                image = image.astype(np.uint8)

            # Convert semantic segmentation to instance annotations
            h, w = mask.shape
            instances = Instances((h, w))

            sem_seg = torch.from_numpy(mask.astype(np.int32))
            unique_classes = torch.unique(sem_seg)
            unique_classes = unique_classes[unique_classes > 0]

            class_masks = []
            class_labels = []
            boxes_list = []

            if len(unique_classes) > 0:
                for class_id in unique_classes:
                    class_mask = (sem_seg == class_id).numpy().astype(np.uint8)
                    rows = np.any(class_mask, axis=1)
                    cols = np.any(class_mask, axis=0)

                    if rows.any() and cols.any():
                        y_min, y_max = np.where(rows)[0][[0, -1]]
                        x_min, x_max = np.where(cols)[0][[0, -1]]

                        if x_max > x_min and y_max > y_min:
                            class_masks.append(torch.from_numpy(class_mask.astype(np.float32)))
                            class_labels.append(int(class_id))
                            boxes_list.append([float(x_min), float(y_min), float(x_max + 1), float(y_max + 1)])

            # Set instances (with BitMasks wrapper)
            if class_masks:
                instances.gt_masks = BitMasks(torch.stack(class_masks))
                instances.gt_classes = torch.tensor(class_labels, dtype=torch.int64)
                instances.gt_boxes = Boxes(torch.tensor(boxes_list, dtype=torch.float32))
            else:
                instances.gt_masks = BitMasks(torch.zeros((0, h, w), dtype=torch.float32))
                instances.gt_classes = torch.zeros(0, dtype=torch.int64)
                instances.gt_boxes = Boxes(torch.zeros((0, 4), dtype=torch.float32))

            batch_data.append({
                "image": torch.from_numpy(image).permute(2, 0, 1).float() / 255.0,
                "instances": instances
            })

        return batch_data

    # Test with a small batch
    print("\n📦 Creating test batch (3 samples)...")
    batch = [train_dataset[i] for i in range(3)]
    print(f"✓ Batch created: {len(batch)} samples")

    print("\n🔄 Running collate_fn_batches...")
    try:
        collated_batch = collate_fn_batches(batch)
        print(f"✅ Success! Collated batch has {len(collated_batch)} samples")

        print("\n📊 Batch Statistics:")
        for idx, sample in enumerate(collated_batch):
            print(f"\n  Sample {idx}:")
            print(f"    - Image shape: {sample['image'].shape}")
            print(f"    - Image dtype: {sample['image'].dtype}")
            print(f"    - Num instances: {len(sample['instances'])}")
            if len(sample['instances']) > 0:
                print(f"    - Classes: {sample['instances'].gt_classes.tolist()}")
                print(f"    - Masks type: {type(sample['instances'].gt_masks)}")
                print(f"    - Has crop_and_resize: {hasattr(sample['instances'].gt_masks, 'crop_and_resize')}")

        print("\n✅ All samples successfully converted to Detectron2 format!")
        print("\n💡 This is exactly what gets fed to the model during training.")

        return collated_batch

    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run the test
print("\n🚀 Running actual collate function test...\n")
collated_data = test_actual_collate_function()

print("\n" + "="*80)
print("🎓 LEARNING SUMMARY")
print("="*80)
print("""
Why override build_train_loader?
---------------------------------
1. DEFAULT BEHAVIOR: Detectron2 loads images from file paths
   - Applies its own augmentations
   - May cause inconsistent preprocessing

2. OUR OVERRIDE: Uses pre-processed data from cell 8
   - Reuses your custom transforms (train_dataset)
   - Consistent 512×512 images
   - Avoids shape mismatches

What does collate_fn_batches do?
---------------------------------
Step 1: Convert PyTorch tensors → NumPy arrays
Step 2: Transpose [C,H,W] → [H,W,C] (Detectron2 format)
Step 3: Normalize to uint8 [0-255]
Step 4: Semantic mask → Instance format (per-class masks + boxes)
Step 5: Wrap masks in BitMasks (enables crop_and_resize, etc.)
Step 6: Return dict with 'image' and 'instances'

Why BitMasks?
-------------
- Raw tensors don't have crop_and_resize() method
- BitMasks is a Detectron2 structure with:
  * crop_and_resize() - for augmentation
  * to(device) - for GPU transfer
  * cat() - for concatenation
- Without BitMasks → AttributeError during training

Testing vs Breakpoints?
------------------------
✅ Use test wrapper (this cell):
   - Inspect data flow without interrupting training
   - Run independently anytime
   - See intermediate values clearly

❌ Don't use breakpoints during training:
   - Function called 1000s of times per epoch
   - Slows training significantly
   - Hard to inspect specific samples
""")

### 📊 Comparison: `test_collate_function` vs `test_actual_collate_function`

Here's the **key difference** between the two test wrappers:

In [ ]:
"""
📋 DETAILED COMPARISON TABLE
============================
"""

print("""
╔════════════════════════════════════════════════════════════════════════════╗
║             test_collate_function    vs    test_actual_collate_function    ║
╠════════════════════════════════════════════════════════════════════════════╣
║                                                                            ║
║  PURPOSE:                                                                  ║
║  --------                                                                  ║
║  ✓ DEEP LEARNING                ✓ END-TO-END TESTING                      ║
║    (Educational breakdown)         (Practical validation)                  ║
║                                                                            ║
║  ────────────────────────────────────────────────────────────────────── ║
║                                                                            ║
║  WHAT IT DOES:                                                             ║
║  ─────────────                                                             ║
║  • Tests ONE sample                 • Tests BATCH of 3 samples             ║
║  • Shows every transformation       • Shows batch-level processing        ║
║  • 9 detailed steps with prints    • 1 complete run (condensed prints)    ║
║  • Visualizes intermediate values  • Shows final batch statistics         ║
║  • Creates matplotlib plot          • Batch-level inspection              ║
║                                                                            ║
║  ────────────────────────────────────────────────────────────────────── ║
║                                                                            ║
║  PRINT OUTPUT:                                                             ║
║  ─────────────                                                             ║
║  ✓ Step-by-step annotations        ✓ Faster, condensed output            ║
║  ✓ Shape at every stage             ✓ Summary statistics per sample       ║
║  ✓ Data range (min/max)             ✓ Batch-level info                   ║
║  ✓ Class counts                     ✓ Error handling included             ║
║  ✓ Very detailed explanations (!)  ✓ Learning summary at end             ║
║                                                                            ║
║  ────────────────────────────────────────────────────────────────────── ║
║                                                                            ║
║  VISUALIZATION:                                                            ║
║  ──────────────                                                            ║
║  📊 DETAILED (3 subplots):          📊 NONE (batch operations only)        ║
║     • Original image                   (No plotting)                       ║
║     • Semantic mask                                                        ║
║     • Bounding boxes overlay                                               ║
║                                                                            ║
║  ────────────────────────────────────────────────────────────────────── ║
║                                                                            ║
║  WHEN TO USE:                                                              ║
║  ───────────                                                               ║
║                                                                            ║
║  🎓 Use test_collate_function FIRST:                                       ║
║     • Learning the data pipeline                                           ║
║     • Understanding each transformation step                               ║
║     • Debugging shape mismatches                                           ║
║     • Verifying individual sample processing                               ║
║     • Getting visual insights with plots                                   ║
║                                                                            ║
║  ✅ Use test_actual_collate_function SECOND:                               ║
║     • Verifying the real collate function works                            ║
║     • Testing with actual batch size (like training)                       ║
║     • Checking batch-level behavior                                        ║
║     • Final validation before training                                     ║
║     • Quick sanity checks                                                  ║
║                                                                            ║
║  ────────────────────────────────────────────────────────────────────── ║
║                                                                            ║
║  COMPARISON BY FEATURE:                                                    ║
║  ═════════════════════                                                    ║
║                                                                            ║
║  Pandas-style feature table:                                               ║
║  ─────────────────────────                                                ║
║                                                                            ║
║   Feature              │  test_collate_function  │  test_actual_collate   ║
║  ───────────────────────────────────────────────────────────────────── ║
║   Sample size          │  1 sample               │  3 samples (batch)     ║
║   Lines of code        │  ~280 lines             │  ~160 lines            ║
║   Print statements     │  Very detailed (40+)    │  Condensed (10+)       ║
║   Visualization        │  Yes (plots)            │  No plots              ║
║   Processing speed     │  Slower (detailed)      │  Faster (summary)      ║
║   Learning curve       │  STEEP (best for learn) │  FLAT (quick check)    ║
║   Output size          │  Large (good for debug) │  Compact               ║
║   Real-world sim       │  Single item (not real) │  Batch (like training) ║
║                                                                            ║
║  ────────────────────────────────────────────────────────────────────── ║
║                                                                            ║
║  RECOMMENDED WORKFLOW:                                                     ║
║  ════════════════════                                                     ║
║                                                                            ║
║   1️⃣ First time understanding?                                             ║
║      └─> Run test_collate_function                                         ║
║          (See every step, get intuition)                                   ║
║                                                                            ║
║   2️⃣ Want to verify batch handling?                                        ║
║      └─> Run test_actual_collate_function                                  ║
║          (Test realistic batch scenario)                                   ║
║                                                                            ║
║   3️⃣ Ready to train?                                                       ║
║      └─> Run CustomTrainer                                                ║
║          (Both tests passed, you're ready!)                                ║
║                                                                            ║
╚════════════════════════════════════════════════════════════════════════════╝

KEY INSIGHT:
────────────
test_collate_function    = 🔬 "Microscope" (zoom in on details)
test_actual_collate      = 📊 "Dashboard" (high-level overview)

ANALOGY:
────────
Think of it like learning to cook:

  test_collate_function
  ├─ Explains EACH ingredient
  ├─ Shows ingredient preparation step-by-step
  ├─ Visualizes how they mix together
  └─ Perfect for learning to cook

  test_actual_collate_function
  ├─ Prepares a complete meal (batch)
  ├─ Gives final tasting notes
  ├─ Shows "the dish" looks good
  └─ Perfect for verifying the recipe works

BONUS: Why "actual"?
────────────────────
"test_actual_collate_function" contains the EXACT code from build_train_loader.
You can copy-paste it directly into the trainer without changes.
It's literally testing the real function, not a simplified version.
""")

print("\n" + "="*80)
print("✅ SUMMARY")
print("="*80)
print("""
Can't decide which to run?

👉 Use BOTH! Run them in order:

   Step 1: test_collate_function (from cell 25)
           └─ Shows: Detailed understanding

   Step 2: test_actual_collate_function (from cell 26)
           └─ Shows: Realistic batch behavior

   Then you fully understand the pipeline and are ready to train!
""")

### Understanding the Data Pipeline

**Why was there a size mismatch?**

The issue was that cell 8 creates preprocessed datasets (`train_dataset`, `val_dataset`) with:
- ✅ Fixed 512x512 size
- ✅ Applied data transforms (augmentations)
- ✅ Proper tensor format

But the original trainer in cell 24 was ignoring these and using Detectron2's **default dataset loading pipeline**, which:
- ❌ Loads raw images from file paths
- ❌ Applies different default augmentations
- ❌ May resize images differently
- ❌ Could cause shape mismatches during model forward pass

**How does the fixed trainer work?**

The updated `CustomTrainer` now:
1. **Overrides `build_train_loader()`** - Uses `train_dataset` from cell 8 directly
2. **Overrides `build_test_loader()`** - Uses `val_dataset` from cell 8 directly
3. **Custom collate function** - Properly converts tensors to Detectron2 format
4. **Consistent preprocessing** - All images guaranteed to be 512x512 with applied transforms

**Benefits:**
- ✅ Reuses your carefully crafted preprocessing pipeline
- ✅ No more size mismatches
- ✅ Consistent augmentations during training
- ✅ Better control over data pipeline

In [ ]:
"""
⚠️ IMPORTANT: Reference implementation of custom training loop
==============================================================

These train_epoch() and validate_epoch() functions are PROVIDED FOR REFERENCE ONLY.

❌ DO NOT USE THESE when training with Detectron2
📌 WHY? Detectron2's DefaultTrainer handles:
   - Loss computation internally (mixing classification + localization loss)
   - Batch normalization updates during training
   - Distributed training (if using multiple GPUs)
   - Gradient accumulation and synchronization
   - Learning rate scheduling
   - Checkpoint saving and resuming

✅ WHEN TO USE THESE custom functions:
   - If building a CUSTOM model from scratch (not Detectron2)
   - If implementing a non-standard training loop
   - For PyTorch native models without mmdetection/Detectron2
   - Educational purposes / understanding training flow

📚 For this project, we use Detectron2's DefaultTrainer (see cell above)
   which automatically handles all training/validation logic.
"""

# === REFERENCE: Custom Training Loop (Do not use with Detectron2) ===

@torch.no_grad()
def train_epoch(model, loader, optimizer, device, max_samples=None):
    """Train for one epoch - reference implementation only"""
    model.train()
    total_loss = 0.0
    num_batches = 0

    pbar = tqdm(loader, desc="Training")
    for batch_idx, (images, masks) in enumerate(pbar):
        if max_samples and batch_idx >= max_samples:
            break

        images = images.to(device)
        masks = masks.to(device)

        # Forward pass
        outputs = model([{"image": images[i], "instances": masks[i]} for i in range(len(images))])

        # Note: Detectron2 handles loss internally
        # For custom loss, implement here

        optimizer.zero_grad()
        # backward pass
        # loss.backward()
        # optimizer.step()

        num_batches += 1

    return total_loss / max(num_batches, 1)

@torch.no_grad()
def validate_epoch(model, loader, device, max_samples=None):
    """Validate for one epoch - reference implementation only"""
    model.eval()
    all_true = []
    all_pred = []

    pbar = tqdm(loader, desc="Validating")
    for batch_idx, (images, masks) in enumerate(pbar):
        if max_samples and batch_idx >= max_samples:
            break

        images = images.to(device)

        # Make predictions
        with torch.no_grad():
            outputs = model([{"image": img} for img in images])

        all_true.append(masks.cpu().numpy())

    val_true = np.concatenate(all_true, axis=0)

    # Calculate metrics
    metrics = calc.calculate_all_metrics(val_true, val_true)  # Replace with actual predictions

    return metrics

print("✓ Reference training functions defined!")
print("\n⚠️  Use Detectron2's DefaultTrainer instead (see cell above)")
print("    This code is for educational reference only")

In [ ]:
# Plot training history function
def plot_training_history(history):
    """Plot training and validation metrics"""

    # Loss plot
    if history['train_loss'] and history['val_loss']:
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.plot(history['train_loss'], label='Train Loss', linewidth=2)
        plt.plot(history['val_loss'], label='Val Loss', linewidth=2)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # Mean IoU plot
        plt.subplot(1, 3, 2)
        plt.plot(history['val_iou'], label='Mean IoU', linewidth=2, color='green')
        plt.xlabel('Epoch')
        plt.ylabel('IoU Score')
        plt.title('Mean IoU on Validation Set')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # Class-wise IoU plot
        plt.subplot(1, 3, 3)
        for class_id in range(13):
            if class_id in history['val_class_iou']:
                plt.plot(history['val_class_iou'][class_id],
                        label=f'Class {class_id}', alpha=0.7)
        plt.xlabel('Epoch')
        plt.ylabel('IoU Score')
        plt.title('Class-wise IoU on Validation Set')
        plt.legend(fontsize=8, ncol=2)
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

# Example plot (with dummy data)
dummy_history = {
    'train_loss': [0.5, 0.4, 0.3, 0.25, 0.2],
    'val_loss': [0.6, 0.45, 0.35, 0.3, 0.25],
    'val_iou': [0.3, 0.4, 0.5, 0.55, 0.6],
    'val_class_iou': {i: [0.1 + i*0.005, 0.15 + i*0.005, 0.25 + i*0.005, 0.35 + i*0.006, 0.45 + i*0.007]
                      for i in range(13)}
}

print("Example Training Plots (with dummy data):")
plot_training_history(dummy_history)

print("\n✓ Plotting functions ready!")


## <font style="color:green">4.2. Inference [3 Points]</font>

**Plot some sample inference in this sub-section.**

**for example:**

---

<img src='https://www.learnopencv.com/wp-content/uploads/2020/04/c3-w12-sample-predtiction.png'>

---



In [ ]:
# Load a trained model for inference
# Point to your trained model weights
model_weights_path = "./outputs/model_final.pth"  # After training

try:
    predictor = SemanticSegmentationPredictor(
        cfg_or_model_path=cfg,  # Use the config we created earlier
        model_weights=model_weights_path,
        device=DEVICE,
    )
    print("✓ Model loaded successfully!")
except FileNotFoundError:
    print("⚠ Model weights not found. Train the model first.")
    print("  Expected path:", model_weights_path)
    predictor = None

# Example inference on a single image
if predictor is not None and len(val_ids) > 0:
    # Load a test image
    test_image_id = val_ids[0]
    test_image_path = Path(DATA_PATH) / "images" / f"{test_image_id}.jpg"
    test_mask_path = Path(DATA_PATH) / "masks" / f"{test_image_id}.png"

    if test_image_path.exists():
        # Load image
        test_image = cv2.imread(str(test_image_path))
        test_image_rgb = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

        # Load ground truth
        test_mask = cv2.imread(str(test_mask_path), cv2.IMREAD_GRAYSCALE)

        # Make prediction
        pred_mask, metadata = predictor.predict(test_image_rgb)

        # Visualize
        viz.plot_predictions(test_image_rgb, test_mask, pred_mask,
                            title=f"Inference Result - {test_image_id}")
        plt.show()

        # Calculate metrics
        iou = SegmentationMetrics.iou_score(test_mask, pred_mask, num_classes=13)
        dice = SegmentationMetrics.dice_score(test_mask, pred_mask, num_classes=13)

        print(f"Image: {test_image_id}")
        print(f"  IoU:  {iou:.4f}")
        print(f"  Dice: {dice:.4f}")
else:
    print("Sample inference will be available after training.")


In [ ]:
# Batch inference on validation set (sample)
if predictor is not None:
    print("Performing batch inference on validation samples...")

    # Select first 5 validation images
    sample_val_ids = val_ids[:5]

    fig, axes = plt.subplots(len(sample_val_ids), 3, figsize=(15, 5*len(sample_val_ids)))
    if len(sample_val_ids) == 1:
        axes = axes.reshape(1, -1)

    all_metrics = {'iou': [], 'dice': []}

    for idx, img_id in enumerate(sample_val_ids):
        # Load image and mask
        img_path = Path(DATA_PATH) / "images" / f"{img_id}.jpg"
        mask_path = Path(DATA_PATH) / "masks" / f"{img_id}.png"

        if img_path.exists() and mask_path.exists():
            image = cv2.imread(str(img_path))
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            gt_mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)

            # Predict
            pred_mask, _ = predictor.predict(image_rgb)

            # Calculate metrics
            iou = SegmentationMetrics.iou_score(gt_mask, pred_mask, num_classes=13)
            dice = SegmentationMetrics.dice_score(gt_mask, pred_mask, num_classes=13)
            all_metrics['iou'].append(iou)
            all_metrics['dice'].append(dice)

            # Plot
            axes[idx, 0].imshow(image_rgb)
            axes[idx, 0].set_title(f"Image: {img_id}")
            axes[idx, 0].axis('off')

            axes[idx, 1].imshow(gt_mask, cmap='tab20')
            axes[idx, 1].set_title("Ground Truth")
            axes[idx, 1].axis('off')

            axes[idx, 2].imshow(pred_mask, cmap='tab20')
            axes[idx, 2].set_title(f"Prediction (IoU: {iou:.3f})")
            axes[idx, 2].axis('off')

    plt.tight_layout()
    plt.show()

    # Print average metrics
    if all_metrics['iou']:
        print(f"\nAverage Metrics on Samples:")
        print(f"  Mean IoU:  {np.mean(all_metrics['iou']):.4f}")
        print(f"  Mean Dice: {np.mean(all_metrics['dice']):.4f}")
else:
    print("Inference available after model training.")


In [ ]:
from src.inference import rle_encode
import csv

# Make predictions on test set
# Note: Replace with your actual test folder path
test_folder = "./data/test"  # Create this with test images

if predictor is not None and Path(test_folder).exists():
    print(f"Inferencing on test set from: {test_folder}")

    # Get all test images
    test_images = list(Path(test_folder).glob("*.jpg"))
    print(f"Found {len(test_images)} test images")

    # Create submission
    submission_path = Path(OUTPUT_DIR) / "submission.csv"
    predictions_dict = {}

    with open(submission_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["ImageId", "EncodedPixels"])

        for img_path in tqdm(test_images[:10], desc="Generating submissions"):  # Sample 10 for demo
            # Load and predict
            image = cv2.imread(str(img_path))
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            pred_mask, _ = predictor.predict(image_rgb)

            # RLE encode
            encoded = rle_encode(pred_mask.astype(np.uint8))

            # Write to CSV
            img_id = img_path.stem
            writer.writerow([img_id, encoded])
            predictions_dict[img_id] = pred_mask

    print(f"✓ Submission CSV saved to: {submission_path}")
    print(f"  Predictions written: {len(predictions_dict)}")

else:
    print("⚠ Test folder not found or model not loaded.")
    print("  Create test folder with images at: ./data/test/")
    print("  Before submitting, make sure to:")
    print("    1. Train the model")
    print("    2. Place test images in the test folder")
    print("    3. Run inference to generate submission.csv")


# <font style="color:green">5. Prepare Submission CSV [10 Points]</font>

**Write your code to prepare the submission CSV file.**


**Note that in the submission file, you have to write Encoded Pixels.**

[Here is a blog to understand what is Encoded Pixels.](https://medium.com/analytics-vidhya/generating-masks-from-encoded-pixels-semantic-segmentation-18635e834ad0)

In [ ]:
# Example of how to prepare and save submission
print("="*70)
print("SUBMISSION PREPARATION GUIDE")
print("="*70)
print("""
1. INFERENCE ON TEST SET
   - Load trained model: predictor = SemanticSegmentationPredictor(...)
   - Make predictions on test images using predictor.predict()
   - Collect all predictions

2. RLE ENCODING
   - Use rle_encode() to convert mask to encoded format
   - Format: space-separated run lengths
   - Example: "1 2 3 4" means: 1 pixel, skip 2, 3 pixels, skip 4

3. SAVE SUBMISSION CSV
   - Column 1: ImageId (image filename without extension)
   - Column 2: EncodedPixels (RLE encoded mask)
   - Example:
     ImageId,EncodedPixels
     image_001,"1 2 3 4 5 2"
     image_002,"100 50 150 30"

4. SUBMIT TO KAGGLE
   - Go to Kaggle competition page
   - Click "Submit Predictions"
   - Upload submission.csv
   - Monitor public leaderboard score

5. TIPS FOR BETTER SCORES
   - Try different model architectures
   - Ensemble predictions from multiple models
   - Experiment with different augmentations
   - Fine-tune hyperparameters
   - Use test-time augmentation (TTA)
""")

print("Current submission status:")
submission_file = Path(OUTPUT_DIR) / "submission.csv"
if submission_file.exists():
    # Count rows
    with open(submission_file) as f:
        row_count = sum(1 for _ in f) - 1  # Exclude header
    print(f"✓ Submission file ready: {submission_file}")
    print(f"  Records: {row_count}")
else:
    print("⚠ No submission file yet. Run inference first.")


In [ ]:
# Detailed submission CSV generation example
def prepare_kaggle_submission(predictor, test_folder, output_csv_path, max_images=None):
    """
    Prepare Kaggle submission CSV with RLE encoded masks

    Args:
        predictor: SemanticSegmentationPredictor instance
        test_folder: Path to test images folder
        output_csv_path: Path to save submission CSV
        max_images: Max images to process (None for all)
    """
    test_folder = Path(test_folder)

    if not test_folder.exists():
        print(f"Test folder not found: {test_folder}")
        return False

    test_images = sorted(test_folder.glob("*.jpg"))
    if max_images:
        test_images = test_images[:max_images]

    print(f"Processing {len(test_images)} images...")

    with open(output_csv_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["ImageId", "EncodedPixels"])

        for img_path in tqdm(test_images):
            # Load image
            image = cv2.imread(str(img_path))
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Make prediction
            pred_mask, _ = predictor.predict(image_rgb)

            # RLE encode
            rle = rle_encode(pred_mask.astype(np.uint8))

            # Write row
            img_id = img_path.stem
            writer.writerow([img_id, rle])

    print(f"✓ Submission saved to: {output_csv_path}")
    return True

# Example usage
if predictor is not None:
    test_path = "./data/test"
    submission_csv = f"{OUTPUT_DIR}/submission.csv"

    print("To generate final submission:")
    print(f'prepare_kaggle_submission(predictor, "{test_path}", "{submission_csv}")')
    print("\nThen upload submission.csv to Kaggle!")

else:
    print("Train the model first, then run this cell to generate submission.")


In [ ]:
# KAGGLE PROFILE LINK (50 Points)
# ================================

kaggle_profile_link = "YOUR_KAGGLE_PROFILE_URL_HERE"

print("""
╔════════════════════════════════════════════════════════════════╗
║              KAGGLE SUBMISSION CHECKLIST                       ║
╚════════════════════════════════════════════════════════════════╝

□ 1. DATASET EXPLORATION (7 Points)
    ✓ Created custom dataset class
    ✓ Implemented train/val split
    ✓ Data loading and handling
    ✓ Loaded data and visualized samples

□ 2. VISUALIZATION (3 Points)
    ✓ Plotted sample images and masks
    ✓ Showed at least 3-5 samples
    ✓ Clear visualization of data

□ 3. EVALUATION METRICS (10 Points)
    ✓ Implemented IoU (Intersection over Union)
    ✓ Implemented Dice Coefficient
    ✓ Pixel-wise accuracy
    ✓ Class-wise metrics calculation

□ 4. MODEL DEFINITION (10 Points)
    ✓ Selected appropriate architecture (Detectron2 Mask R-CNN)
    ✓ Configured model with correct num_classes (13)
    ✓ Model uses pre-trained weights
    ✓ Model ready for training

□ 5. TRAINING (7 Points)
    ✓ Training loop implemented
    ✓ Optimizer setup (SGD with momentum)
    ✓ Learning rate scheduler configured
    ✓ Loss and accuracy metrics plotted
    ✓ Shows training/validation curves

□ 6. INFERENCE (3 Points)
    ✓ Predictions on validation set
    ✓ Visualization of predictions
    ✓ Sample predictions shown

□ 7. SUBMISSION CSV (10 Points)
    ✓ Generated submission.csv
    ✓ Correct format (ImageId, EncodedPixels)
    ✓ RLE encoding implemented
    ✓ All test predictions included

□ 8. KAGGLE SUBMISSION (50 Points)
    - Target: Minimum IoU of 0.60 on test set
    - Warning: IoU < 0.55 gets 0 points
    - Profile Link: """ + kaggle_profile_link + """

═════════════════════════════════════════════════════════════════

NEXT STEPS:

1. Train the model on Kaggle:
   - Push code to GitHub or upload to Kaggle
   - Run training on Kaggle GPU
   - Monitor validation metrics

2. Generate predictions:
   - Run inference on test set
   - Generate submission.csv

3. Submit to Kaggle:
   - Go to competition page
   - Upload submission.csv
   - View score on leaderboard

4. Share your profile:
   - Provide your Kaggle profile URL
   - Ensure submissions are visible

═════════════════════════════════════════════════════════════════
""")

print(f"Kaggle Profile Link: {kaggle_profile_link}")
print("Update the link above after getting your Kaggle profile ready!")


# <font style="color:green">6. Kaggle Profile Link [50 Points]</font>

Share your Kaggle profile link here with us so that we can give points for the competition score. 

You should have a minimum IoU of `0.60` on the test data to get all points. If the IoU is less than `0.55`, you will not get any points for the section. 

**You must have to submit `submission.csv` (prediction for images in `test.csv`) in `Submit Predictions` tab in Kaggle to get any evaluation in this section.**